# Segmenting and Clustering Neighborhoods in New York City & London

In [1]:
#install packages
!pip install numpy
!pip install pandas
!pip install requests
!pip install bs4
!pip install plotly
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes
!pip install html5lib
!pip install OSGridConverter
print('Packages installed.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.1

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.1

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.

Packages installed.


In [2]:
#import libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
import requests
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from geopy.geocoders import Nominatim
from OSGridConverter import grid2latlong
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
print('Libraries imported.')

Libraries imported.


## Set Up Foursquare Credentials and Venue Functions

In [3]:
#input Foursquare credentials
CLIENT_ID = 'YTE1O5DN3TRLBFJR2V4CMOU0SNG2AF3XQA0CGFA4KA4SFAOK' # your Foursquare ID
CLIENT_SECRET = 'SRMIKFL2VBEBVO3ADRTIEDTRAB5OPAS4HOPTF00PXTOVXWT3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [4]:
#function to get the top 100 venues that for given neighborhood within a radius of 500 meters

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
#             boro,
            lat, 
            lng, 
            v['venue']['name'], 
#             v['venue']['city'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [5]:
#function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## NYC : Download Data from IBM Server and Transform into Pandas Dataframe 

In [6]:
#download NYC dataset from IBM server
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json

In [7]:
#load data 
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [8]:
#get a feel for how the data is structured
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

In [9]:
#all the relevant data is located within "features"
nyc_neighborhoods_data = newyork_data['features']

In [10]:
# define the dataframe columns
column_names = ['City','Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe (creating an empty dataframe)
nyc_neighborhoods = pd.DataFrame(columns=column_names)

In [11]:
#transform data into pandas dataframe
for data in nyc_neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    city = 'New York City'
    
    nyc_neighborhoods = nyc_neighborhoods.append({'City': city, 'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [12]:
#populated dataframe with nyc data 
nyc_neighborhoods.loc[nyc_neighborhoods['Neighborhood'] == "Howland Hook"]
nyc_neighborhoods.drop(index=257, inplace=True)
nyc_neighborhoods.head()

,City,Borough,Neighborhood,Latitude,Longitude
0,New York City,Bronx,Wakefield,40.894705,-73.847201
1,New York City,Bronx,Co-op City,40.874294,-73.829939
2,New York City,Bronx,Eastchester,40.887556,-73.827806
3,New York City,Bronx,Fieldston,40.895437,-73.905643
4,New York City,Bronx,Riverdale,40.890834,-73.912585


In [13]:
#merge neighborhood and borough into one column
nyc_neighborhoods['Neighborhood'] = nyc_neighborhoods['Neighborhood'] + " (" + nyc_neighborhoods['Borough'] + ")"
nyc_neighborhoods.drop(columns = 'Borough', inplace=True)
nyc_neighborhoods.head()

,City,Neighborhood,Latitude,Longitude
0,New York City,Wakefield (Bronx),40.894705,-73.847201
1,New York City,Co-op City (Bronx),40.874294,-73.829939
2,New York City,Eastchester (Bronx),40.887556,-73.827806
3,New York City,Fieldston (Bronx),40.895437,-73.905643
4,New York City,Riverdale (Bronx),40.890834,-73.912585


In [14]:
#number of columns and rows
nyc_neighborhoods.shape

(305, 4)

## London: Webscrape Data from Wiki Page and Transform into Pandas Dataframe

In [15]:
#download webpage and save text in the html_data variable
london_url = "https://en.wikipedia.org/wiki/List_of_areas_of_London" 
html_data = requests.get(london_url).text

In [16]:
#parse through the html data
soup = BeautifulSoup(html_data,"html.parser")

In [17]:
#find all tables
tables = soup.find_all('table')

In [18]:
#create empty list to store cleansed data
table_contents=[]

#extract the html data and assign it to the corresponding column
row_num = 0
for row in tables[1].find("tbody").find_all("tr"):
    if row_num > 0: #want to skip the first row as that contains a heading (refine later so that there's no need to skip row) 
        cell = {} #create a dictionary to hold all record values
        col = row.find_all('td')
        remove_tail_borough = col[1].text.split('[')
        cell['Borough']= remove_tail_borough[0]
        cell['Neighborhood'] = col[0].text
        cell['PostTown'] = col[2].text
        cell['OSGridRef'] = col[5].text.replace('\n','')
        if cell['OSGridRef'] == '': #use the OS grid to find the coordinates
            cell['Latitude'] = 0
            cell['Longitude'] = 0 
        else:
            l=grid2latlong(cell['OSGridRef']) 
            cell['Latitude'] = l.latitude 
            cell['Longitude'] = l.longitude
        table_contents.append(cell) #consolidate all dictionaries into a list 
    row_num = row_num + 1
df=pd.DataFrame(table_contents) #convert list into dataframe

#print dataframe
df.head()

,Borough,Neighborhood,PostTown,OSGridRef,Latitude,Longitude
0,"Bexley, Greenwich",Abbey Wood,LONDON,TQ465785,51.486484,0.109318
1,"Ealing, Hammersmith and Fulham",Acton,LONDON,TQ205805,51.510591,-0.264585
2,Croydon,Addington,CROYDON,TQ375645,51.362934,-0.025780
3,Croydon,Addiscombe,CROYDON,TQ345665,51.381625,-0.068126
4,Bexley,Albany Park,"BEXLEY, SIDCUP",TQ478728,51.434929,0.125663


In [19]:
#number of columns and rows
df.shape

(531, 6)

In [20]:
#filter results for PostTown with London only (records with multiple PostTowns will NOT be included)
london_neighborhoods = df[df['PostTown'] == 'LONDON'].reset_index(drop=True) 
london_neighborhoods.head()

,Borough,Neighborhood,PostTown,OSGridRef,Latitude,Longitude
0,"Bexley, Greenwich",Abbey Wood,LONDON,TQ465785,51.486484,0.109318
1,"Ealing, Hammersmith and Fulham",Acton,LONDON,TQ205805,51.510591,-0.264585
2,City,Aldgate,LONDON,TQ334813,51.514885,-0.078356
3,Westminster,Aldwych,LONDON,TQ307810,51.512819,-0.117388
4,Bromley,Anerley,LONDON,TQ345695,51.408585,-0.066989


In [21]:
#number of columns and rows
london_neighborhoods.shape

(297, 6)

In [22]:
#format the London dataframe to match the NYC dataframe so that we can merge them

london_neighborhoods.drop(['PostTown'], axis=1, inplace = True)#drop PostTown column since all of them are London
london_neighborhoods.drop(['OSGridRef'], axis=1, inplace = True)#drop OSGridRef since we already extracted the latitude and longitude
london_neighborhoods['City'] = 'London' #add city column to help distinguish between NYC and London
cols = london_neighborhoods.columns.tolist() #current order of columns 
cols = cols[-1:] + cols[:-1] #move City column to the front
london_neighborhoods[cols].head()

,City,Borough,Neighborhood,Latitude,Longitude
0,London,"Bexley, Greenwich",Abbey Wood,51.486484,0.109318
1,London,"Ealing, Hammersmith and Fulham",Acton,51.510591,-0.264585
2,London,City,Aldgate,51.514885,-0.078356
3,London,Westminster,Aldwych,51.512819,-0.117388
4,London,Bromley,Anerley,51.408585,-0.066989


In [23]:
#merge neighborhood and borough into one column
london_neighborhoods['Neighborhood'] = london_neighborhoods['Neighborhood'] + " (" + london_neighborhoods['Borough'] + ")"
london_neighborhoods.drop(columns = 'Borough', inplace=True)
london_neighborhoods.head()

,Neighborhood,Latitude,Longitude,City
0,"Abbey Wood (Bexley, Greenwich )",51.486484,0.109318,London
1,"Acton (Ealing, Hammersmith and Fulham)",51.510591,-0.264585,London
2,Aldgate (City),51.514885,-0.078356,London
3,Aldwych (Westminster),51.512819,-0.117388,London
4,Anerley (Bromley),51.408585,-0.066989,London


In [24]:
#verify the number of columns and rows
london_neighborhoods.shape

(297, 4)

In [25]:
#merge the NYC dataframe and London dataframe 
nyc_london_neighborhoods = pd.concat([nyc_neighborhoods, london_neighborhoods]).reset_index(drop=True) 
nyc_london_neighborhoods.tail()

,City,Neighborhood,Latitude,Longitude
597,London,Wood Green (Haringey),51.598237,-0.116745
598,London,Woodford (Redbridge),51.604820,0.028068
599,London,Woodside Park (Barnet),51.617324,-0.186791
600,London,Woolwich (Greenwich),51.496238,0.066504
601,London,Wormwood Scrubs (Hammersmith and Fulham),51.519148,-0.235411


In [26]:
#verify the number of columns and rows
nyc_london_neighborhoods.shape

(602, 4)

## Use Folium Map to Visualize Neighborhoods in NYC and London

In [27]:
#obtain geographic coordinates of NYC
nyc_address = 'New York City, NY'

nyc_geolocator = Nominatim(user_agent="ny_explorer")
nyc_location = nyc_geolocator.geocode(nyc_address)
nyc_latitude = nyc_location.latitude
nyc_longitude = nyc_location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(nyc_latitude, nyc_longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [28]:
#obtain geographic coordinates of London
london_address = 'London, United Kingdom'

london_geolocator = Nominatim(user_agent="uk_explorer")
london_location = london_geolocator.geocode(london_address)
london_latitude = london_location.latitude
london_longitude = london_location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(london_latitude, london_longitude))

The geograpical coordinate of London are 51.5073219, -0.1276474.


In [29]:
#find the midpoint so that the map shows both cities

list_latitudes = [nyc_latitude, london_latitude]
list_longitudes = [nyc_longitude, london_longitude]

avg_latitude = sum(list_latitudes)/len(list_latitudes)
avg_longitude = sum(list_longitudes)/len(list_longitudes)

print('The geographic coordinate of the midpoint between NYC and London are {}, {}.'.format(avg_latitude, avg_longitude))

The geographic coordinate of the midpoint between NYC and London are 46.110025, -37.0668313.


In [30]:
# create map that includes both NYC and London using the midpoint latitude and longitude values
map_nyc_london = folium.Map(location=[avg_latitude, avg_longitude], zoom_start=4)

# add markers to map
for lat, lng, neighborhood in zip(nyc_london_neighborhoods['Latitude'], nyc_london_neighborhoods['Longitude'], nyc_london_neighborhoods['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc', fill_opacity=0.7, 
        parse_html=False).add_to(map_nyc_london)
                                           
map_nyc_london

## Use Foursquare to Identify Top Venues in Each Neighborhood

In [31]:
#run the above function on each neighborhood
nyc_london_venues = getNearbyVenues(names=nyc_london_neighborhoods['Neighborhood'],
                                   latitudes=nyc_london_neighborhoods['Latitude'],
                                   longitudes=nyc_london_neighborhoods['Longitude']
                                  )

Wakefield (Bronx)
Co-op City (Bronx)
Eastchester (Bronx)
Fieldston (Bronx)
Riverdale (Bronx)
Kingsbridge (Bronx)
Marble Hill (Manhattan)
Woodlawn (Bronx)
Norwood (Bronx)
Williamsbridge (Bronx)
Baychester (Bronx)
Pelham Parkway (Bronx)
City Island (Bronx)
Bedford Park (Bronx)
University Heights (Bronx)
Morris Heights (Bronx)
Fordham (Bronx)
East Tremont (Bronx)
West Farms (Bronx)
High  Bridge (Bronx)
Melrose (Bronx)
Mott Haven (Bronx)
Port Morris (Bronx)
Longwood (Bronx)
Hunts Point (Bronx)
Morrisania (Bronx)
Soundview (Bronx)
Clason Point (Bronx)
Throgs Neck (Bronx)
Country Club (Bronx)
Parkchester (Bronx)
Westchester Square (Bronx)
Van Nest (Bronx)
Morris Park (Bronx)
Belmont (Bronx)
Spuyten Duyvil (Bronx)
North Riverdale (Bronx)
Pelham Bay (Bronx)
Schuylerville (Bronx)
Edgewater Park (Bronx)
Castle Hill (Bronx)
Olinville (Bronx)
Pelham Gardens (Bronx)
Concourse (Bronx)
Unionport (Bronx)
Edenwald (Bronx)
Bay Ridge (Brooklyn)
Bensonhurst (Brooklyn)
Sunset Park (Brooklyn)
Greenpoint (Br

In [32]:
nyc_london_venues.shape

(17903, 7)

In [33]:
#group the venues by neighborhood
nyc_london_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Abbey Wood (Bexley, Greenwich )",4,4,4,4,4,4
"Acton (Ealing, Hammersmith and Fulham)",12,12,12,12,12,12
Aldgate (City),100,100,100,100,100,100
Aldwych (Westminster),100,100,100,100,100,100
Allerton (Bronx),31,31,31,31,31,31
Anerley (Bromley),5,5,5,5,5,5
Angel (Islington),15,15,15,15,15,15
Annadale (Staten Island),9,9,9,9,9,9
Archway (Islington),25,25,25,25,25,25


In [34]:
#unique categories for venues
print('There are {} uniques categories.'.format(len(nyc_london_venues['Venue Category'].unique())))

There are 496 uniques categories.


In [35]:
#Analyze Each Neighborhood

# one hot encoding
nyc_london_onehot = pd.get_dummies(nyc_london_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
nyc_london_onehot['Neighborhood']= nyc_london_venues['Neighborhood']
neighborhood_col_index = nyc_london_onehot.columns.get_loc("Neighborhood")

#move neighborhood column to the first column 
fixed_columns = [nyc_london_onehot.columns[neighborhood_col_index]] + list(nyc_london_onehot.columns[0:neighborhood_col_index]) + list(nyc_london_onehot.columns[neighborhood_col_index + 1:])
nyc_london_onehot = nyc_london_onehot[fixed_columns]


print("The unique venue categories have now become columns.") 
print("Therefore the column count should equal the number of unique categories:", len(nyc_london_onehot.columns))


nyc_london_onehot.head()

The unique venue categories have now become columns.
Therefore the column count should equal the number of unique categories: 496


,Neighborhood,Acai House,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Betting Shop,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Casino,Castle,Caucasian Restaurant,Cemetery,Cha Chaan Teng,Champagne Bar,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Cigkofte Place,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Residence Hall,College Science Building,College Stadium,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cooking School,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cricket Ground,Cuban Restaurant,Cupcake Shop,Currency Exchange,Cycle Studio,Czech Restaurant,Dam,Dance Studio,Daycare,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gelato Shop,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Sto

In [36]:
#the venue dataframe is now expanded to include the different categories
nyc_london_onehot.shape

(17903, 496)

In [37]:
#group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
nyc_london_grouped = nyc_london_onehot.groupby('Neighborhood').mean().reset_index()
nyc_london_grouped.head()

,Neighborhood,Acai House,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Betting Shop,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Casino,Castle,Caucasian Restaurant,Cemetery,Cha Chaan Teng,Champagne Bar,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Cigkofte Place,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Residence Hall,College Science Building,College Stadium,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cooking School,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cricket Ground,Cuban Restaurant,Cupcake Shop,Currency Exchange,Cycle Studio,Czech Restaurant,Dam,Dance Studio,Daycare,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gelato Shop,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Sto

In [38]:
nyc_london_grouped.shape

(602, 496)

In [39]:
#create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
nyc_london_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
nyc_london_neighborhoods_venues_sorted['Neighborhood'] = nyc_london_grouped['Neighborhood']

for ind in np.arange(nyc_london_grouped.shape[0]):
    nyc_london_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(nyc_london_grouped.iloc[ind, :], num_top_venues)

nyc_london_neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Abbey Wood (Bexley, Greenwich )",Fried Chicken Joint,Playground,Newsagent,Grocery Store,Filipino Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Fabric Shop
1,"Acton (Ealing, Hammersmith and Fulham)",Pub,Gym / Fitness Center,Supermarket,Bakery,Park,Grocery Store,Wine Bar,Bus Stop,Train Station,Exhibit
2,Aldgate (City),Coffee Shop,Gym / Fitness Center,Hotel,Cocktail Bar,Indian Restaurant,Italian Restaurant,Café,Middle Eastern Restaurant,Pizza Place,Sushi Restaurant
3,Aldwych (Westminster),Theater,Coffee Shop,Pub,French Restaurant,Restaurant,Sandwich Place,Hotel,Cocktail Bar,Italian Restaurant,Spanish Restaurant
4,Allerton (Bronx),Pizza Place,Deli / Bodega,Supermarket,Discount Store,Bakery,Chinese Restaurant,Pharmacy,Grocery Store,Martial Arts School,Electronics Store


In [40]:
nyc_london_neighborhoods_venues_sorted.shape

(602, 11)

## Use K-Means to Cluster Neighborhoods Across NYC and London

In [41]:
# set number of clusters
kclusters = 5

nyc_london_grouped_clustering = nyc_london_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nyc_london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 4, 4, 2, 0, 4, 0, 1], dtype=int32)

In [42]:
#quick visual on how the clusters are distributed
print(kmeans.labels_)

[0 0 0 4 4 2 0 4 0 1 1 1 4 4 4 4 4 0 0 4 0 0 4 0 4 4 4 4 4 4 2 0 4 0 4 4 0
 4 4 4 4 4 0 4 2 0 0 0 0 0 0 4 1 0 4 4 0 0 0 3 0 4 4 4 4 4 4 0 0 0 0 4 4 4
 4 1 4 0 4 1 0 4 0 0 4 4 0 2 1 4 4 0 4 4 2 4 0 4 4 4 0 4 2 1 4 4 4 0 0 0 0
 4 4 4 0 4 2 0 4 4 4 4 4 0 4 0 0 0 4 4 4 4 4 4 1 0 2 0 4 0 3 4 4 0 0 0 0 4
 4 4 4 4 2 4 2 0 0 3 0 4 0 4 0 4 4 0 4 4 4 4 0 4 4 0 4 0 4 4 0 4 4 4 4 0 4
 4 0 0 0 4 4 4 4 0 4 4 0 0 4 4 4 4 0 1 4 0 0 0 4 4 4 0 4 4 0 0 4 0 4 4 1 4
 1 4 4 4 4 0 4 0 0 3 0 0 0 2 4 0 4 3 0 0 2 0 0 0 4 0 0 4 4 0 0 4 4 0 0 0 4
 4 0 4 0 4 2 0 4 0 4 4 4 4 4 0 0 4 4 3 4 0 0 4 0 0 4 4 2 0 4 4 4 0 4 0 0 0
 4 2 0 0 4 4 0 0 0 0 0 4 4 0 0 4 4 0 4 4 0 4 4 0 3 4 4 4 4 4 0 3 4 4 4 0 4
 4 4 1 4 0 4 4 3 4 4 4 0 4 0 4 0 0 4 4 4 4 0 4 3 4 4 4 4 0 1 3 1 0 4 4 0 4
 1 4 0 0 4 0 4 0 0 4 4 0 4 0 0 4 0 4 4 4 0 4 4 4 0 0 4 0 4 0 1 4 4 4 0 0 4
 4 4 0 0 0 0 4 4 4 4 4 4 1 0 4 4 4 4 0 0 4 4 4 3 0 4 1 4 4 4 4 0 4 4 4 4 3
 3 1 4 4 4 4 4 3 4 0 1 4 3 0 0 0 0 4 0 0 4 4 4 4 4 4 4 0 2 4 3 0 4 4 0 2 4
 0 0 0 2 0 0 0 4 4 0 4 0 

In [43]:
# add clustering labels
nyc_london_neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

merged = nyc_london_neighborhoods

merged = merged.join(nyc_london_neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merged.head() # check the last columns!

,City,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,New York City,Wakefield (Bronx),40.894705,-73.847201,4,Pharmacy,Laundromat,Ice Cream Shop,Sandwich Place,Dessert Shop,Deli / Bodega,Caribbean Restaurant,Donut Shop,Zoo Exhibit,Field
1,New York City,Co-op City (Bronx),40.874294,-73.829939,4,Bus Station,Restaurant,Discount Store,Trail,Grocery Store,Pizza Place,Donut Shop,Park,Fast Food Restaurant,Post Office
2,New York City,Eastchester (Bronx),40.887556,-73.827806,4,Bus Station,Caribbean Restaurant,Deli / Bodega,Diner,Juice Bar,Seafood Restaurant,Donut Shop,Platform,Pizza Place,Bowling Alley
3,New York City,Fieldston (Bronx),40.895437,-73.905643,4,River,Plaza,Bus Station,Music Venue,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory
4,New York City,Riverdale (Bronx),40.890834,-73.912585,4,Bus Station,Playground,Bank,Gym,Park,Plaza,Zoo Exhibit,Fast Food Restaurant,Farmers Market,Farm


In [44]:
# create map
map_clusters = folium.Map(location=[avg_latitude, avg_longitude], zoom_start=4)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Neighborhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

In [45]:
merged.loc[merged['Cluster Labels'] == 0, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,Edenwald (Bronx),Supermarket,Athletics & Sports,Sandwich Place,Grocery Store,Farm,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Film Studio
181,Floral Park (Queens),Indian Restaurant,Grocery Store,Basketball Court,Playground,Salon / Barbershop,Zoo Exhibit,Farmers Market,Farm,Falafel Restaurant,Factory
220,Sunnyside (Staten Island),Spa,Market,Grocery Store,Gym,Café,American Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm
251,Richmond Town (Staten Island),Café,Bagel Shop,Italian Restaurant,Spa,Event Space,Event Service,Exhibit,Fabric Shop,Film Studio,Factory
254,Concord (Staten Island),Deli / Bodega,Peruvian Restaurant,Grocery Store,Coffee Shop,Thrift / Vintage Store,Athletics & Sports,Train Station,Fabric Shop,Factory,Field
291,Lighthouse Hill (Staten Island),Spa,Trail,Italian Restaurant,Art Museum,Café,Falafel Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm
305,"Abbey Wood (Bexley, Greenwich )",Fried Chicken Joint,Playground,Newsagent,Grocery Store,Filipino Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Fabric Shop
306,"Acton (Ealing, Hammersmith and Fulham)",Pub,Gym / Fitness Center,Supermarket,Bakery,Park,Grocery Store,Wine Bar,Bus Stop,Train Station,Exhibit
307,Aldgate (City),Coffee Shop,Gym / Fitness Center,Hotel,Cocktail Bar,Indian Restaurant,Italian Restaurant,Café,Middle Eastern Restaurant,Pizza Place,Sushi Restaurant
310,Angel (Islington),Grocery Store,Fast Food Restaurant,Tram Station,Bakery,Park,Diner,Pub,Café,Indian Restaurant,Zoo Exhibit


In [46]:
merged.loc[merged['Cluster Labels'] == 1, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
198,New Brighton (Staten Island),Bus Stop,Park,Deli / Bodega,Bridal Shop,Daycare,Playground,Discount Store,Bowling Alley,Zoo Exhibit,Field
205,Port Richmond (Staten Island),Martial Arts School,Bus Stop,Rental Car Location,Food Truck,Zoo Exhibit,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory
224,Park Hill (Staten Island),Athletics & Sports,Bus Stop,Park,Hotel,Coffee Shop,Farm,Field,Fast Food Restaurant,Farmers Market,Factory
226,Graniteville (Staten Island),Boat or Ferry,Bus Stop,Grocery Store,Fish Market,Event Space,Exhibit,Fabric Shop,Factory,Falafel Restaurant,Farm
227,Arlington (Staten Island),Intersection,Bus Stop,American Restaurant,Deli / Bodega,Boat or Ferry,Coffee Shop,Farm,Film Studio,Filipino Restaurant,Field
229,Grasmere (Staten Island),Bus Stop,Bagel Shop,Bank,Restaurant,Italian Restaurant,Ice Cream Shop,Grocery Store,Nail Salon,Food,Park
238,Butler Manor (Staten Island),Baseball Field,Pool,Bus Stop,Gas Station,Falafel Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm,Zoo Exhibit
241,Arden Heights (Staten Island),Pizza Place,Bus Stop,Pharmacy,Coffee Shop,Zoo Exhibit,Field,Ethiopian Restaurant,Event Service,Event Space,Exhibit
245,Bloomfield (Staten Island),Burger Joint,Theme Park,Recreation Center,Bus Stop,Farm,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Factory
284,Willowbrook (Staten Island),Bus Stop,Zoo Exhibit,Fish & Chips Shop,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Factory,Falafel Restaurant


In [47]:
merged.loc[merged['Cluster Labels'] == 2, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Clason Point (Bronx),Park,Scenic Lookout,Convenience Store,Pool,Playground,South American Restaurant,Boat or Ferry,Grocery Store,Bus Stop,Factory
81,Dyker Heights (Brooklyn),Park,Burger Joint,Bagel Shop,Playground,Zoo Exhibit,Event Service,Event Space,Exhibit,Fabric Shop,Factory
91,Bergen Beach (Brooklyn),Harbor / Marina,Baseball Field,Playground,Park,Athletics & Sports,Hockey Field,Event Space,Exhibit,Field,Fabric Shop
148,South Ozone Park (Queens),Park,Bar,Deli / Bodega,Food Truck,Food,Hotel,Donut Shop,Sandwich Place,Grocery Store,Fast Food Restaurant
192,Somerville (Queens),Park,Zoo Exhibit,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Factory,Falafel Restaurant,Farm
203,Todt Hill (Staten Island),Park,Trail,Zoo Exhibit,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Factory,Falafel Restaurant
244,Chelsea (Staten Island),Arts & Crafts Store,Park,Sandwich Place,Ice Cream Shop,Steakhouse,Farm,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
274,Stuyvesant Town (Manhattan),Boat or Ferry,Park,Pet Service,Bistro,Farmers Market,Coffee Shop,Gas Station,Heliport,Bar,Harbor / Marina
302,Bayswater (Queens),Playground,Zoo Exhibit,Film Studio,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Factory,Falafel Restaurant
309,Anerley (Bromley),Grocery Store,Train Station,Hardware Store,Park,Food Truck,Fast Food Restaurant,Ethiopian Restaurant,Event Service,Fountain,Event Space


In [48]:
merged.loc[merged['Cluster Labels'] == 3, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
85,Sea Gate (Brooklyn),IT Services,Bus Station,Beach,American Restaurant,Spa,Farm,Film Studio,Filipino Restaurant,Field,Fast Food Restaurant
172,Breezy Point (Queens),Beach,Trail,Bus Stop,Food Truck,Monument / Landmark,Falafel Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm
178,Rockaway Beach (Queens),Beach,Latin American Restaurant,Fast Food Restaurant,Food Stand,BBQ Joint,Metro Station,Seafood Restaurant,Arepa Restaurant,Ice Cream Shop,Moving Target
179,Neponsit (Queens),Beach,Zoo Exhibit,Film Studio,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Factory,Falafel Restaurant
183,Jamaica Estates (Queens),Intersection,Indian Restaurant,Dog Run,Falafel Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm,Zoo Exhibit,Film Studio
191,Rockaway Park (Queens),Beach,Mediterranean Restaurant,Diner,Steakhouse,Smoke Shop,Pharmacy,Bar,Bank,Bagel Shop,Seafood Restaurant
202,Grymes Hill (Staten Island),Dog Run,Zoo Exhibit,Film Studio,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Factory,Falafel Restaurant
204,South Beach (Staten Island),Beach,Pier,Bus Stop,Theme Park,Deli / Bodega,Athletics & Sports,Event Space,Exhibit,Fabric Shop,Filipino Restaurant
232,Midland Beach (Staten Island),Beach,Bookstore,Restaurant,Pet Store,Bus Stop,Liquor Store,Chinese Restaurant,Deli / Bodega,Russian Restaurant,Farm
256,Randall Manor (Staten Island),Beach,Bus Stop,Bagel Shop,Pizza Place,Zoo Exhibit,Film Studio,Event Space,Exhibit,Fabric Shop,Factory


In [49]:
merged.loc[merged['Cluster Labels'] == 4, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield (Bronx),Pharmacy,Laundromat,Ice Cream Shop,Sandwich Place,Dessert Shop,Deli / Bodega,Caribbean Restaurant,Donut Shop,Zoo Exhibit,Field
1,Co-op City (Bronx),Bus Station,Restaurant,Discount Store,Trail,Grocery Store,Pizza Place,Donut Shop,Park,Fast Food Restaurant,Post Office
2,Eastchester (Bronx),Bus Station,Caribbean Restaurant,Deli / Bodega,Diner,Juice Bar,Seafood Restaurant,Donut Shop,Platform,Pizza Place,Bowling Alley
3,Fieldston (Bronx),River,Plaza,Bus Station,Music Venue,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory
4,Riverdale (Bronx),Bus Station,Playground,Bank,Gym,Park,Plaza,Zoo Exhibit,Fast Food Restaurant,Farmers Market,Farm
5,Kingsbridge (Bronx),Pizza Place,Sandwich Place,Bar,Mexican Restaurant,Latin American Restaurant,Deli / Bodega,Mobile Phone Shop,Café,Spanish Restaurant,Supermarket
6,Marble Hill (Manhattan),Sandwich Place,Department Store,Seafood Restaurant,Clothing Store,Pharmacy,Discount Store,Diner,Coffee Shop,Supplement Shop,Bakery
7,Woodlawn (Bronx),Pub,Pizza Place,Food Truck,Food & Drink Shop,Playground,Deli / Bodega,Park,Grocery Store,Train Station,Donut Shop
8,Norwood (Bronx),Park,Pizza Place,Bank,Mobile Phone Shop,Pharmacy,Bus Station,Spanish Restaurant,Fast Food Restaurant,Burger Joint,Sandwich Place
9,Williamsbridge (Bronx),Bar,Nightclub,Caribbean Restaurant,Deli / Bodega,Exhibit,Fabric Shop,Factory,Event Space,Event Service,Falafel Restaurant
